In [88]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1530808215043_0015,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1530808215043_0015,spark,idle,Link,Link,✔


In [115]:
// The Kafka broker hosts and topic used to write to Kafka
val kafkaBrokers="wn0-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092,wn1-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092"
val kafkaTopic="rawevents"
val zookeepers = "zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181"

// Import bits useed for declaring schemas and working with JSON data
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

// Define a schema for the data
val schema = (new StructType).
add("house_id", StringType).
add("timestamp", StringType).
add("value", StringType).
add("household_id", StringType)

var kafkastreamPerHourPerDaySchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("Date", DateType).
add("LastHour", IntegerType).
add("Id", StringType).
add("TotalConsumptionForThatHourAndDay",DoubleType).
add("window",(new StructType).
    add("start",DateType).
    add("end",DateType)
   )

var kafkastreamPerHourSchema = (new StructType).
add("house_id", StringType).
add("household_id", StringType).
add("LastHour", StringType).
add("Id", StringType).
add("MeanConsumptionForThatHour",DoubleType).
add("SDConsumptionForThatHour",DoubleType).
add("1SDConsumption", DoubleType).
add("window",(new StructType).
    add("start",DateType).
    add("end",DateType)
   )

kafkastreamPerHourSchema: org.apache.spark.sql.types.StructType = StructType(StructField(house_id,StringType,true), StructField(household_id,StringType,true), StructField(LastHour,StringType,true), StructField(Id,StringType,true), StructField(MeanConsumptionForThatHour,DoubleType,true), StructField(SDConsumptionForThatHour,DoubleType,true), StructField(1SDConsumption,DoubleType,true), StructField(window,StructType(StructField(start,DateType,true), StructField(end,DateType,true)),true))

In [97]:
var commonDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets","earliest").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), schema).alias("consumption")).
select("consumption.*").
selectExpr(
    "house_id",
    "household_id",
    "cast(cast(timestamp as int) as timestamp) as timestamp",
    "cast(value as double) as value",
    "hour(from_unixtime(timestamp)) as Lasthour",
    "from_unixtime(timestamp) as FullDateTime",
    "cast(from_unixtime(timestamp) as date) as Date"
).
withColumn("Id",concat(col("house_id"),lit("_"),col("household_id"),lit("_"),col("Date"),lit("_"),col("Lasthour"))).
withWatermark("timestamp","10 minute").
dropDuplicates("Id")

commonDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [house_id: string, household_id: string ... 6 more fields]

In [111]:
var streamPerHourPerDay = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("household_id"),col("Date"),col("Lasthour"),col("Id")
).
agg(
    sum("value").as("TotalConsumptionForThatHourAndDay")
).
selectExpr("cast(Id as string) as key","to_json(struct(*)) AS value").as[(String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHourPerDay").
option("topic", "streamPerHourPerDay").
option("failOnDataLoss","false").
outputMode("append").
start()

streamPerHourPerDay: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@227ea717

In [114]:
var streamPerHour = commonDF.
groupBy(
    window($"timestamp", "1 hour", "1 hour"),
    col("house_id"),col("household_id"),col("Lasthour")
).
agg(
    mean("value").as("MeanConsumptionForThatHour")
    ,stddev_pop("value").as("SDConsumptionForThatHour")
).
withColumn(
    "1SDConsumption"
    ,col("MeanConsumptionForThatHour") + col("SDConsumptionForThatHour")
).
withColumn("Id",concat(col("house_id"),lit("_"),col("household_id"),lit("_"),col("Lasthour"))).
selectExpr("cast(Id as string) as key","to_json(struct(*)) AS value").as[(String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointstreamPerHour").
option("topic", "streamPerHour").
option("failOnDataLoss","false").
outputMode("update").
start()

streamPerHour: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@21061a9d

In [124]:
var kafkastreamPerHourPerDayDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHourPerDay").
option("startingOffsets","latest").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourPerDaySchema).alias("streamPerHourPerDay")).
select("streamPerHourPerDay.*").
select(
    col("house_id"),
    col("household_id"),
    col("Lasthour"),
    col("Id"),
    col("window.start").as("Start"),
    col("TotalConsumptionForThatHourAndDay")
)

var kafkastreamPerHourDF = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", "streamPerHour").
option("startingOffsets","latest").
load().
selectExpr("CAST(key AS STRING) AS key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), kafkastreamPerHourSchema).alias("streamPerHour")).
select("streamPerHour.*").
select(
    col("house_id"),
    col("household_id"),
    col("Lasthour"),
    col("window.start").as("Start"),
    col("window.end").as("End"),
    col("1SDConsumption"),
    col("MeanConsumptionForThatHour"),
    col("SDConsumptionForThatHour")
)

var finalResult = kafkastreamPerHourPerDayDF.as("x").join(
    kafkastreamPerHourDF.as("y"),
    expr("""
        x.house_id = y.house_id AND
        x.household_id = y.household_id AND
        x.Lasthour = y.Lasthour AND
        x.Start >= y.Start AND x.Start < y.End AND
        TotalConsumptionForThatHourAndDay > 1SDConsumption
        """)
).
select(col("x.Id"))

finalResult: org.apache.spark.sql.DataFrame = [Id: string]

In [125]:
var query = finalResult.
writeStream.
format("csv").
option("path","/example/AlertType1").
option("checkpointLocation", "/path/chkpointFinalResult").
outputMode("append").
start() 

org.apache.spark.sql.AnalysisException: Inner join between two streaming DataFrames/Datasets is not supported;;
Join Inner, ((((house_id#1625 = house_id#1675) && (household_id#1626 = household_id#1676)) && (Lasthour#1628 = cast(Lasthour#1677 as int))) && (((Start#1640 >= Start#1692) && (Start#1640 < End#1693)) && (TotalConsumptionForThatHourAndDay#1630 > 1SDConsumption#1681)))
:- SubqueryAlias x
:  +- Project [house_id#1625, household_id#1626, Lasthour#1628, Id#1629, window#1631.start AS Start#1640, TotalConsumptionForThatHourAndDay#1630]
:     +- Project [streamPerHourPerDay#1622.house_id AS house_id#1625, streamPerHourPerDay#1622.household_id AS household_id#1626, streamPerHourPerDay#1622.Date AS Date#1627, streamPerHourPerDay#1622.LastHour AS LastHour#1628, streamPerHourPerDay#1622.Id AS Id#1629, streamPerHourPerDay#1622.TotalConsumptionForThatHourAndDay AS TotalConsumptionForThatHourAndDay#1630, streamPerHourPerDay#1622.window AS window#1631]
:        +- Project [jsontostructs(Stru

In [91]:
query.status

<console>:33: error: not found: value streamPerHourPerDay
       streamPerHourPerDay.stop
       ^


In [83]:
%%bash
hdfs dfs -ls /example/csvdata

Found 10 items
drwxr-xr-x   - livy supergroup          0 2018-07-06 15:39 /example/csvdata/_spark_metadata
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00000-978a5c09-8efa-4d89-9083-93122c8f68bd-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00001-b5fa8eda-94d9-4eae-a34b-6ea24c8bfadd-c000.csv
-rw-r--r--   1 livy supergroup        100 2018-07-06 15:39 /example/csvdata/part-00002-6c325cad-b52a-48f6-9d61-409cfd4ab6c8-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00003-59e418e9-62c6-44ef-aa4a-7f40dadb772e-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00004-1a275171-c400-4e30-8675-cbdac9e7c4bd-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00005-fde47d31-5882-4008-8751-5ba55bc493b3-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 15:39 /example/csvdata/part-00006-0a9c3d59

In [85]:
%%bash
hdfs dfs -cat /example/csvdata/part-00002-6c325cad-b52a-48f6-9d61-409cfd4ab6c8-c000.csv

0,0,2013-09-02T13:00:00.000Z,10045.438999999997,13,2013-09-02 13:00:00,2013-09-02,0_0_2013-09-02_13


In [109]:
%%bash
hdfs dfs -rm -r /path/chkpoint
hdfs dfs -rm -r /example/AlertType1

rm: `/path/chkpoint': No such file or directory
rm: `/example/AlertType1': No such file or directory
